# Question 1


Considérons un scénario où un joueur lance un dé. <br>
On note X le résultat du dé, ainsi X suit une loi discrète uniforme entre 1 et 6. <br>
Ensuite, il dispose de x lancés de dé supplémentaire afin de faire un maximum de 6. <br>
On note Y le nombre de 6 obtenus, ainsi Y suit une loi binomial de paramètre (x, 1/6) <br>
On utilise la square loss. <br>
$ $
\begin{equation}
f^* = E[Y|X]
\end{equation}

\begin{equation}
\begin{split}
R(f^*) & = E_{(X,Y)~p}[l(Y, f^*(X))] \\
& = E_X[E_Y[l(y, f^*(x)) | X = x]] \\
& = E_X[E_Y[(y-\frac{x}{6})^2 | X = x]] \\
& = E_X[\sum_{y = 0}^x (y-\frac{x}{6})^2 \binom{x}{y} (\frac{1}{6})^y (\frac{5}{6})^{x-y}]\\
& = \sum_{x=1}^{6}(\sum_{y = 0}^x (y-\frac{x}{6})^2 \binom{x}{y} (\frac{1}{6})^y (\frac{5}{6})^{x-y})P(X=x) \\
& = \sum_{x=1}^{6}(\sum_{y = 0}^x (y-\frac{x}{6})^2 \binom{x}{y} (\frac{1}{6})^y (\frac{5}{6})^{x-y})\frac{1}{6}
\end{split}
\end{equation}
$$

Faisons ce calcul en utilisant python

In [30]:
import math

x_sum = 0
for x in range(1, 6+1):
    y_sum = 0
    for y in range(0, x + 1):
        y_sum += (y - x/6)**2 * math.comb(x, y) * (1/6)**y * (5/6)**(x-y)
    x_sum += y_sum * (1/6)
print(f"Bayes risk : {x_sum}")

Bayes risk : 0.4861111111111111


# Question 2 : Bayes estimator and Bayes risk

* Rappel des paramètres de l'expérience:
    * X est uniformément distribué entre 1 et 6
    * Y suit une loi binomial de paramètres (X, 1/6)
    * On utilise la squared loss
    
    
* Le risk de Bayes théorique calculé est 0.486

In [31]:
import numpy as np

rng = np.random.default_rng()

In [32]:
# define the loss function
def loss(x, y):
    return (y - x)**2

In [33]:
# generate the data
from typing import Tuple


def generate_data(n_samples: int) -> Tuple[np.array, np.array]:
    """
    n_samples: number of samples to generate
    return: X, Y
    """
    X = rng.integers(1, 7, n_samples)
    Y = rng.binomial(X, 1/6)
    return X, Y

In [34]:
# define the bayes predictor
def bayes_predictor(X: np.array) -> np.array:
    """
    It generates the prediction for the bayes predictor which the expected value of Y given X (in the square loss case).
    Y is a binomial law so the expected value is np, here n = X and p = 1/6
    """
    return X/6

In [35]:
# define a bad estimator to compare results
def bad_estimator(X: np.array) -> np.array:
    """
    X: np.array of inputs
    Given a set of inputs, return a bad estimator
    """

    return rng.integers(0, 6, size=X.shape[0]) # random estimator

In [36]:
# define the empirical risk function
def empirical_risk(estimator, loss, X: np.array, Y: np.array) -> float:
    """
    estimator: function that takes X and returns Y
    loss: function that takes two values and returns a loss
    X: np.array of inputs
    Y: np.array of outputs
    Given a set of inputs and outputs, return the empirical risk
    """

    Y_pred = estimator(X)
    return np.mean(loss(Y, Y_pred))

### Run the simulation

In [37]:
X, Y = generate_data(int(1e6))

print("Empirical risk for the bayes predictor: ", empirical_risk(bayes_predictor, loss, X, Y))
print("Empirical risk for the bad estimator: ", empirical_risk(bad_estimator, loss, X, Y))

Empirical risk for the bayes predictor:  0.4870844166666667
Empirical risk for the bad estimator:  7.16226


On retrouve bien un risque empirique plus faible pour le prédicteur de bayes qu'un prédicteur random et on retrouve bien la valeur théorique de 0.48 attendue